## Scraping Multiple Pages - Colorado

#### Importing Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.parse
import pandas as pd

#### Scraping the data

In [2]:
address = []
bedrooms = []
bathrooms = []
sqft = []
year_built = []
parking = []
pool = []
price = []

# Url part I
url_1 = 'https://www.trulia.com'

for i in range(1, 26):
    
    #website
    www = 'https://www.trulia.com/PA/Philadelphia/' +str(i) +'_p/'
    
    #requests
    response = requests.get(www)
    
    # soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    #result container
    result_container = soup.find_all('li', {'class':'SearchResultsList__WideCell-b7y9ki-2'})
    
    #update results
    results_update = []
    
    # only results with attribute 'data-testeid'
    for r in result_container:
        if r.has_attr('data-testid'):
            results_update.append(r)
        
    #relative url
    relative_url = []
    
    #loop through results
    for i in results_update:
        for j in i.find_all('div', {'data-testid':'property-card-details'}):
            relative_url.append(j.find('a').get('href'))
            
    #empty list url_joined
    url_joined = []

    for i in relative_url:
        url_joined.append(urllib.parse.urljoin(url_1, i))
        
       
    # loop through all joined links
    for i in url_joined:
        response = requests.get(i)

        #create soup object
        soup = BeautifulSoup(response.content, 'html.parser')

        #address
        try:
            address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
        except:
            address.append('')

        #bedrooms
        try:
            bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
        except:
            bedrooms.append('')

        #bathrooms
        try:
            bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
        except:
            bathrooms.append('')

        #sqft
        try:
            sqft.append(soup.find('li', {'data-testid':'floor'}).get_text())
        except:
            sqft.append('')

        #year built
        try:
            year_built.append(soup.find('div', string='Year Built').findNext('div').get_text())
        except:
            year_built.append('')

         #parking
        try:
            parking.append(soup.find('div', string='Parking').findNext('div').get_text())
        except:
            parking.append('')

         #pool
        try:
            pool.append(soup.find('div', string='Pool').findNext('div').get_text())
        except:
            pool.append('')

         #price
        try:
            price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
        except:
            price.append('')

        #creating a dictionary with the results
        output = {'Address': address, 'Bedrooms':bedrooms, 'Bathrooms': bathrooms, 'Area': sqft, 'Year Built': year_built,
                 'Parking': parking, 'Pool': pool, 'Price': price}

In [3]:
df = pd.DataFrame(output)
df

,Address,Bedrooms,Bathrooms,Area,Year Built,Parking,Pool,Price
0,8025 Walker St,4 Beds,2 Baths,"1,428 sqft",1945,Open Parking,No,"$290,000"
1,3910 N 9th St,3 Beds,2 Baths,"1,024 sqft",1920,Open Parking,Pool,"$179,000"
2,819 Kendrick St,3 Beds,3 Baths,"1,572 sqft",1966,1 Car Garage,No,"$359,900"
3,611 E Locust Ave,6 Beds,4 Baths,"3,240 sqft",1925,Open Parking,No,"$350,000"
4,524 Lombard St #C,3 Beds,3 Baths,"1,536 sqft",1978,1 Car Garage,Pool,"$589,000"
...,...,...,...,...,...,...,...,...
995,3149 N 15th St,4 Beds,,"1,480 sqft",1920,Open Parking,Pool,"$180,000"
996,1232 N Front St #2A,2 Beds,2 Baths,"1,179 sqft (on 0.27 acres)",2018,1 Car Garage,Pool,"$474,000"
997,4642 Weymouth St,2 Beds,1 Bath,800 sqft,1930,Open Parking,Pool,"$125,000"
998,1010 Arch St #404,2 Beds,1 Bath,860 sqft,1900,None,Pool,"$285,000"


In [4]:
df['Location'] = 'Philadelphia'
df

,Address,Bedrooms,Bathrooms,Area,Year Built,Parking,Pool,Price,Location
0,8025 Walker St,4 Beds,2 Baths,"1,428 sqft",1945,Open Parking,No,"$290,000",Philadelphia
1,3910 N 9th St,3 Beds,2 Baths,"1,024 sqft",1920,Open Parking,Pool,"$179,000",Philadelphia
2,819 Kendrick St,3 Beds,3 Baths,"1,572 sqft",1966,1 Car Garage,No,"$359,900",Philadelphia
3,611 E Locust Ave,6 Beds,4 Baths,"3,240 sqft",1925,Open Parking,No,"$350,000",Philadelphia
4,524 Lombard St #C,3 Beds,3 Baths,"1,536 sqft",1978,1 Car Garage,Pool,"$589,000",Philadelphia
...,...,...,...,...,...,...,...,...,...
995,3149 N 15th St,4 Beds,,"1,480 sqft",1920,Open Parking,Pool,"$180,000",Philadelphia
996,1232 N Front St #2A,2 Beds,2 Baths,"1,179 sqft (on 0.27 acres)",2018,1 Car Garage,Pool,"$474,000",Philadelphia
997,4642 Weymouth St,2 Beds,1 Bath,800 sqft,1930,Open Parking,Pool,"$125,000",Philadelphia
998,1010 Arch St #404,2 Beds,1 Bath,860 sqft,1900,None,Pool,"$285,000",Philadelphia


In [5]:
df.to_csv('df_philadelphia.csv', index=False)